In [1]:
# imports
from configs import config
from configs.detection import detect_people
from scipy.spatial import distance as dist 
import numpy as np
import argparse
import imutils
import cv2
import os


In [2]:
FLAGS = []

if __name__ == '__main__':
    parser = argparse.ArgumentParser()

    
    parser.add_argument('-w', '--weights',
        default='./weights/yolov3.weights')

    parser.add_argument('-cfg', '--config',
        default='./yolo-coco/yolov3.cfg')

    parser.add_argument('-v', '--video-path',
        default='cctv.mp4')

    parser.add_argument('-vo', '--video-output-path',
        default='output.avi')

    parser.add_argument('-d', '--display', 
        default=1)
    parser.add_argument('-l', '--labels',
        default='./yolo-coco/coco.names')


    FLAGS, unparsed = parser.parse_known_args()


# Get the labels
    LABELS = open(FLAGS.labels).read().strip().split('\n')


# Load the weights and configutation to form the pretrained YOLOv3 model
    net = cv2.dnn.readNetFromDarknet(FLAGS.config, FLAGS.weights)

    # Get the output layer names of the model
    ln = net.getLayerNames()
    ln = [ln[i - 1] for i in net.getUnconnectedOutLayers()]

    

In [ ]:

if FLAGS.video_path:
# initialize the video stream and pointer to output video file
    print("[INFO] accessing video stream...")
# open input video if available else webcam stream
    vs = cv2.VideoCapture(FLAGS.video_path if FLAGS.video_path else 0)
    writer = None

    # loop over the frames from the video stream
    while True:
        # read the next frame from the input video
        (grabbed, frame) = vs.read()

        # if the frame was not grabbed, then that's the end fo the stream 
        if not grabbed:
            break

        # resize the frame and then detect people (only people) in it
        frame = imutils.resize(frame, width=700)
        results = detect_people(frame, net, ln, personIdx=LABELS.index("person"))

        # initialize the set of indexes that violate the minimum social distance
        violate = set()

        # ensure there are at least two people detections (required in order to compute the
        # the pairwise distance maps)
        if len(results) >= 2:
            # extract all centroids from the results and compute the Euclidean distances
            # between all pairs of the centroids
            centroids = np.array([r[2] for r in results])
            D = dist.cdist(centroids, centroids, metric="euclidean")

            # loop over the upper triangular of the distance matrix
            for i in range(0, D.shape[0]):
                for j in range(i+1, D.shape[1]):
                    # check to see if the distance between any two centroid pairs is less
                    # than the configured number of pixels
                    if D[i, j] < config.MIN_DISTANCE:
                        # update the violation set with the indexes of the centroid pairs
                        violate.add(i)
                        violate.add(j)

        # loop over the results
        for (i, (prob, bbox, centroid)) in enumerate(results):
            # extract teh bounding box and centroid coordinates, then initialize the color of the annotation
            (startX, startY, endX, endY) = bbox
            (cX, cY) = centroid
            color = (0, 255, 0)

            # if the index pair exists within the violation set, then update the color
            if i in violate:
                color = (0, 0, 255)

            # draw (1) a bounding box around the person and (2) the centroid coordinates of the person
            cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
            cv2.circle(frame, (cX, cY), 5, color, 1)

        # draw the total number of social distancing violations on the output frame
        text = " Violations: {}".format(len(violate))
        cv2.putText(frame, text, (10, frame.shape[0] - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 3)
        
        if len(violate)>0:
            import winsound
            frequency = 2000  # Set Frequency To 2500 Hertz
            duration = 500  # Set Duration To 1000 ms == 1 second
            winsound.Beep(frequency, duration)
            

        # check to see if the output frame should be displayed to the screen
        if FLAGS.display > 0:
            # show the output frame
            cv2.imshow("Output", frame)
            key = cv2.waitKey(1) & 0xFF

            # if the 'q' key is pressed, break from the loop
            if key == ord("q"):
                break

        # if  the video writer has not been  as none
        if writer is None:
            # initialize the video writer
            fourcc = cv2.VideoWriter_fourcc(*"MJPG")
            writer = cv2.VideoWriter(FLAGS.video_output_path, fourcc, 25, (frame.shape[1], frame.shape[0]), True)

        # if the video writer is not None, write the frame to the output video file
        if writer is not None:
            print("[INFO] writing stream to output")
            writer.write(frame)



[INFO] accessing video stream...
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] 

[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
[INFO] writing stream to output
